In [1]:
# Installing Libraries


# ! pip install transformers -q
# ! pip install annoy -q

# Importing Libraries


# from transformers import AutoTokenizer, AutoModel
# import torch
import pandas as pd
import numpy as np
import string
import re
import spacy
from tqdm.auto import tqdm
tqdm.pandas()

import pickle

# Checking to see if CUDA is available if so storing details in device


# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Define Varaiables and Functions

In [2]:
nums = [
        '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
        '31', '32', '33', '34', '35', '36', '37', '38', '39', '40'
        ]

two_letter_words = [
  'aa',	# rough cindery lava
  'ab', #	an abdominal muscle
  'ad', #	an advertisement
  'ag', #	S. African a general exclamation
  'ah', #	an exclamation
  'ai', #	the three-toed sloth
  'am', #	the present tense of be
  'an', #	a form of the indefinite article
  'as', #	used to convey relative extent or degree
  'at', #	expressing location or time
  'aw', #	an exclamation
  'ax', #	variant spelling of axe
  'ay', #	variant spelling of aye
  'ba', #	the soul, in Egyptian mythology
  'be', #	to exist
  'bi', #	bisexual
  'bo', #	a kind of fig tree
  'by', #	beside
  'da', #	a person's father
  'do', #	to perform an action
  'dy', #	a type of sediment
  'ee', #	dialect form of oh
  'eh', #	an exclamation
  'el', #	an elevated railway
  'em', #	a measuring unit in printing
  'en', #	a measuring unit in printing
  'er', #	expressing doubt or hesitation
  'ex', #	a former spouse or partner
  'fa', #	a musical note
  'ga', #	a people in Ghana
  'gi', #	judo jacket
  'go', #	to move or travel
  'ha', #	an exclamation
  'he', #	a male person or animal
  'hi', #	a greeting
  'ho', #	an exclamation
  'id', #	a part of the mind
  'if', #	introducing a conditional clause
  'in', #	within
  'io', #	a North American moth
  'is', #	the present tense of be
  'it', #	a thing
  'ja', #	S. African yes
  'jo', #	Scottish, archaic a sweetheart
  'ju', #	a kind of Chinese pottery
  'ka', #	the spirit, in Egyptian mythology
  'ki', #	a plant of the lily family
  'la', #	a musical note
  'li', #	a Chinese unit of distance
  'lo', #	archaic used to draw attention to something
  'ma', #	a person's mother
  'me', #	the objective case of the first-person pronoun I
  'mi', #	a musical note
  'mo', #	a moment
  'mu', #	a Greek letter
  'my', #	belonging to me
  'no', #	not any
  'nu', #	a Greek letter
  'ob', #	a type of gene
  'od', #	a supposed natural force
  'of', #	belonging to
  'og', #	Australian, archaic a shilling
  'oh', #	an exclamation
  'oi', #	used to attract attention
  'ok', #	used to express assent, agreement, etc.
  'om', #	a sacred mantra
  'on', #	supported by or covering
  'op', #	an operation
  'or', #	used to link alternatives
  'os', #	a bone
  'ou', #	a Hawaiian bird
  'ow', #	expressing pain
  'ox', #	a cow or bull
  'oy', #	= oi
  'oz', #	Australia(n)
  'pa', #	father
  'pi', #	a Greek letter
  'po', #	a chamber pot
  'qi', #	the life force, in Chinese philosophy
  'ra', #	a moraine, in Norway or Sweden
  're', # a musical note
  'ri', #	a Japanese unit of length
  'se', #	a Chinese musical instrument
  'si', #	= te
  'so', #	therefore
  'ta', #	thank you
  'te', #	a musical note
  'ti', #	= te
  'to', #	expressing direction in relation to a location
  'uh', #	expressing hesitation
  'um', #	expressing hesitation
  'up', #	towards a higher position
  'us', #	the objective case of we
  'wa', #	a people on the borders of China and Burma
  'we', #	oneself and other people
  'wu', #	a dialect of Chinese
  'xi', #	a Greek letter
  'xu', #	a Vietnamese unit of money
  'ye', #	archaic the plural form of thou
  'yi', #	a people in parts of China
  'yo', #	used as a greeting
  'yu', #	an ancient Chinese wine container
]

In [3]:
def lower(text):
  return text.lower()

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  # text = text.translate(str.maketrans('', ' ', string.punctuation))
  # return text

  res = re.sub(r'[^\w\s]', '', text)
  return res

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

# remove multiple spaces
def remove_mul_space(text):
  return " ".join(text.split())

def clean(text):
  
  text = lower(text)
  text = remove_urls(text)
  text = remove_nonascii(text)
  text = remove_punctuations(text)
  text = remove_num(text)
  text = remove_mul_space(text)

  return text

In [4]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_word_cloud(dictionary):
  text = []
  for key in dictionary:
    for i in range(dictionary[key]):
      text.append(key)

  result = ' '.join(text)

  wordcloud = WordCloud(width=2400, height=1600, background_color='white', collocations=False).generate(result)

  plt.figure(figsize=(20,10))
  plt.imshow(wordcloud, interpolation = 'bilinear')
  plt.axis('off')

In [5]:
load_model = spacy.load('en', disable = ['parser','ner'])

def lemmatize_dic_keys(dictionary):

  for key in list(dictionary):

    old_key = key
    doc = load_model(old_key)
    new_key = " ".join([token.lemma_ for token in doc])

    if new_key != old_key:
      if new_key in dictionary.keys():
        # print(True, key)
        dictionary[new_key] += dictionary[old_key]
        del dictionary[old_key]
        # print(dictionary)

      else:
        # print(False, key)
        dictionary[new_key] = dictionary[old_key]
        del dictionary[old_key]
        # print(dictionary)


  return dictionary

# Read Data

In [6]:
'''
  This is to Read Fake News articles 
'''


bus_fake, edu_fake, ent_fake, pol_fake, sports_fake, tech_fake = [], [], [], [], [], []


for num in nums:

  bus = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Business/biz" + num +  ".fake.txt"
  edu = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Education/edu" + num + ".fake.txt"
  ent = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Enterntainment/entmt" + num + ".fake.txt"
  pol = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Politics/polit"+ num + ".fake.txt"
  spo = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Sports/sports" + num + ".fake.txt"
  tech = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/fake/Technology/tech0" + num + ".fake.txt"
  
  # Business Fake News
  with open(bus, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    bus_fake.append(result)

  # Education Fake News
  with open(edu, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    edu_fake.append(result)

  # Entertainment Fake News
  with open(ent, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    ent_fake.append(result)

  # Political Fake News
  with open(pol, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    pol_fake.append(result)

  # Sports Fake News
  with open(spo, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    sports_fake.append(result)

  # Technology based Fake News
  with open(tech, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    tech_fake.append(result)

In [7]:
'''
  This is to Read Legit News articles 
'''


bus_real, edu_real, ent_real, pol_real, sports_real, tech_real = [], [], [], [], [], []

for num in nums:

  bus = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Business/biz" + num +  ".legit.txt"
  edu = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Education/edu" + num + ".legit.txt"
  ent = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Entertainment/entmt" + num + ".legit.txt"
  pol = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Politics/polit"+ num + ".legit.txt"
  spo = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Sports/sports" + num + ".legit.txt"
  tech = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/fakeNewsDatasets/fakeNewsDataset/legit/Technology/tech0" + num + ".legit.txt"
  
  # Business Real News
  with open(bus, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    bus_real.append(result)

  # Education Real News
  with open(edu, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    edu_real.append(result)

  # Entertainment Real News
  with open(ent, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    ent_real.append(result)

  # Political Real News
  with open(pol, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    pol_real.append(result)

  # Sports Real News
  with open(spo, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    sports_real.append(result)

  # Technology based Real News
  with open(tech, "r") as f:
    result = ""
    mylist = f.read().splitlines() 
    for line in mylist:
      if line != '':
        result += line+". "
    tech_real.append(result)

In [8]:
from collections import Counter

def get_words(fake_data, real_data, n):

  for i in range(len(fake_data)):
    fake_data[i] = clean(fake_data[i])
  for i in range(len(real_data)):
    real_data[i] = clean(real_data[i])

  fake_words, true_words = [], []
  for item in fake_data:
    fake_words.extend(item.strip().split())
  for item in real_data:
    true_words.extend(item.strip().split())

  # fake_words = [ word for word in fake_words if len(word) > 2 or (word == 'a' or word in two_letter_words) ]
  # true_words = [ word for word in true_words if len(word) > 2 or (word == 'a' or word in two_letter_words) ]

  fake_words = [ word for word in fake_words if len(word) >= 3 ]
  true_words = [ word for word in true_words if len(word) >= 3 ]

  counts_common = {}
  counts_true = {}
  counts_fake = {}

  counts_fake = dict(Counter(fake_words))

  counts_true = dict(Counter(true_words))

  for key in counts_true.keys():
    if key in counts_fake.keys():
      counts_common[key] = counts_true[key]+counts_fake[key]

  for key in counts_true.keys():
    counts_true[key] = counts_true[key]

  for key in counts_fake.keys():
    counts_fake[key] = counts_fake[key]

  counts_fake = lemmatize_dic_keys(counts_fake)
  counts_true = lemmatize_dic_keys(counts_true)

  fake_keys = counts_fake.keys()
  true_keys = counts_true.keys()

  common_keys = []

  for item in fake_keys:
    if item in true_keys:
      common_keys.append(item)

  for k in common_keys:
      counts_fake.pop(k, None)
      counts_true.pop(k, None)

  l1, l2 = len(list(counts_fake.keys())), len(list(counts_true.keys()))

  n = min(l1,l2,n)

  true = dict(sorted(counts_true.items(), key = lambda item: item[1], reverse = True)[:n])
  fake = dict(sorted(counts_fake.items(), key = lambda item: item[1], reverse = True)[:n])

  if '-PRON-' in fake.keys():
    del fake['-PRON-']
  if '-PRON-' in true.keys():
    del true['-PRON-']

  return list(fake.keys()), list(true.keys())

# Analyzing

In [9]:
business_fake, business_real = get_words(fake_data = bus_fake, real_data = bus_real, n = 100)
education_fake, education_real = get_words(fake_data = edu_fake, real_data = edu_real, n = 100)
sport_fake, sport_real = get_words(fake_data = sports_fake, real_data = sports_real, n = 100)
entertainment_fake, entertainment_real = get_words(fake_data = ent_fake, real_data = ent_real, n = 100)
political_fake, political_real = get_words(fake_data = pol_fake, real_data = pol_real, n = 100)
technology_fake, technology_real = get_words(fake_data = tech_fake, real_data = tech_real, n = 100)

In [11]:
fake = [business_fake, education_fake, sport_fake, entertainment_fake, political_fake, technology_fake]
real = [business_real, education_real, sport_real, entertainment_real, political_real, technology_real]

In [15]:
words = {
    'business': {
        'fake': business_fake,
        'true': business_real
    },
    'education': {
        'fake': education_fake,
        'true': education_real
    },
    'sport': {
        'fake': sport_fake,
        'true': sport_real
    },
    'entertainment':{
        'fake': entertainment_fake,
        'true': entertainment_real
    },
    'political':{
        'fake': political_fake,
        'true': political_real
    },
    'technology':{
        'fake': technology_fake,
        'true': technology_real
    }
}

In [18]:
with open(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/wordlist.pickle", "wb") as output_file:
  pickle.dump(words, output_file)

In [ ]:
fake_unique, true_unique = [], []

for item in fake:
  for word in item:
    if word not in fake_unique:
      fake_unique.append(word)

for item in real:
  for word in item:
    if word not in fake_unique:
      if word not in true_unique:
        true_unique.append(word)
    else:
      fake_unique.remove(word)

In [ ]:
len(fake_unique)

484

In [ ]:
len(true_unique)

514

In [ ]:
words = {
    'fake': fake_unique,
    'true': true_unique
}

In [ ]:
with open(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/wordlist.pickle", "wb") as output_file:
  pickle.dump(words, output_file)